In [1]:
import math
import hess
import matplotlib.pyplot as plt
import numpy as np
from hess.nets import SubLayerLinear, SubNetLinear, MaskedLayerLinear, MaskedNetLinear
import hess.net_utils as net_utils
import torch
from torch import nn

In [ ]:
n_hidden = 5
width = 1024

In [6]:
model = simple_net(2, 1, n_layers=5, k=1024)

In [7]:
hess.net_utils.set_model_prune_rate(model, 0.5)

==> Setting prune rate of network to 0.3
==> Setting prune rate of sequential.0 to 0.3
==> Setting prune rate of sequential.2 to 0.3
==> Setting prune rate of sequential.4 to 0.3
==> Setting prune rate of sequential.6 to 0.3
==> Setting prune rate of sequential.8 to 0.3
==> Setting prune rate of sequential.10 to 0.3
==> Setting prune rate of sequential.12 to 0.3


In [8]:
hess.net_utils.freeze_model_weights(model)

=> Freezing model weights
==> No gradient to sequential.0.weight
==> No gradient to sequential.0.bias
==> No gradient to sequential.2.weight
==> No gradient to sequential.2.bias
==> No gradient to sequential.4.weight
==> No gradient to sequential.4.bias
==> No gradient to sequential.6.weight
==> No gradient to sequential.6.bias
==> No gradient to sequential.8.weight
==> No gradient to sequential.8.bias
==> No gradient to sequential.10.weight
==> No gradient to sequential.10.bias
==> No gradient to sequential.12.weight
==> No gradient to sequential.12.bias


In [9]:
def twospirals(n_points, noise=.5, random_state=920):
    """
     Returns the two spirals dataset.
    """
    n = np.sqrt(np.random.rand(n_points,1)) * 600 * (2*np.pi)/360
    d1x = -1.5*np.cos(n)*n + np.random.randn(n_points,1) * noise
    d1y =  1.5*np.sin(n)*n + np.random.randn(n_points,1) * noise
    return (np.vstack((np.hstack((d1x,d1y)),np.hstack((-d1x,-d1y)))),
            np.hstack((np.zeros(n_points),np.ones(n_points))))

In [10]:
X, Y = twospirals(500, noise=1.3)
train_x = torch.FloatTensor(X)
train_y = torch.FloatTensor(Y).unsqueeze(-1)

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_func = torch.nn.BCEWithLogitsLoss()


for step in range(1000):
    optimizer.zero_grad()
    outputs = model(train_x)

    loss=loss_func(outputs,train_y)
    print(loss)
    loss.backward()
    optimizer.step()

tensor(0.6932, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.6935, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.7078, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(4.3978, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.6883, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.6935, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.6934, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.6931, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.6928, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


KeyboardInterrupt: 